In [6]:
%region us-east-2
%glue_version "3.0"
%number_of_workers 10
%worker_type "G.2X"
%extra_jars s3://sftp-pipeline-bucket/lib/geomesa-fs-spark-runtime_2.12-3.5.0.jar

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.37.4 and you have 0.37.2 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Previous region: None
Setting new region to: us-east-2
Region is set to: us-east-2
Setting Glue version to: 3.0
Previous number of workers: 5
Setting new number of workers to: 10
Previous worker type: G.1X
Setting new worker type to: G.2X
Extra jars to be included:
s3://sftp-pipeline-bucket/lib/geomesa-fs-spark-runtime_2.12-3.5.0.jar
s3://sftp-pipeline-bucket/lib/geomesa-fs-spark-runtime_2.12-3.5.0.jar


In [8]:
%%configure
{
    "--conf": "spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.kryo.registrator=org.locationtech.geomesa.spark.GeoMesaSparkKryoRegistrator",
    "--enable-spark-ui": "true",
    "--spark-event-logs-path": "s3://sftp-pipeline-clean/sparkLogs"
}

The following configurations have been updated: {'--conf': 'spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.kryo.registrator=org.locationtech.geomesa.spark.GeoMesaSparkKryoRegistrator', '--enable-spark-ui': 'true', '--spark-event-logs-path': 's3://sftp-pipeline-clean/sparkLogs'}


In [1]:
import org.locationtech.geomesa.spark.jts._
spark.withJTS

Authenticating with profile=default
glue_role_arn retrieved from profile: arn:aws:iam::121891376456:role/sftp-pipeline-glue-role
Trying to create a Glue session for the kernel.
Worker Type: G.2X
Number of Workers: 10
Session ID: d2834edc-85d2-4735-b622-347d1b9b65a5
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.2
--session-language scala
--enable-glue-datacatalog true
--extra-jars s3://sftp-pipeline-bucket/lib/geomesa-fs-spark-runtime_2.12-3.5.0.jar
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.kryo.registrator=org.locationtech.geomesa.spark.GeoMesaSparkKryoRegistrator
--enable-spark-ui true
--spark-event-logs-path s3://sftp-pipeline-clean/sparkLogs
Waiting for session d2834edc-85d2-4735-b622-347d1b9b65a5 to get into ready status...
Session d2834edc-85d2-4735-b622-347d1b9b65a5 has been created

import org.locationtech.geomesa.spark.jts._
res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@

In [2]:
val planet = spark.read.orc("s3://osm-pds/planet/planet-latest.orc")

planet: org.apache.spark.sql.DataFrame = [id: bigint, type: string ... 11 more fields]



In [3]:
planet.printSchema()

root
 |-- id: long (nullable = true)
 |-- type: string (nullable = true)
 |-- tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- lat: decimal(9,7) (nullable = true)
 |-- lon: decimal(10,7) (nullable = true)
 |-- nds: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ref: long (nullable = true)
 |-- members: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- ref: long (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- changeset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- uid: long (nullable = true)
 |-- user: string (nullable = true)
 |-- version: long (nullable = true)
 |-- visible: boolean (nullable = true)




In [4]:
planet.show()

+---+----+--------------------+-----------+-----------+---+-------+---------+-------------------+--------+------------------+-------+-------+
| id|type|                tags|        lat|        lon|nds|members|changeset|          timestamp|     uid|              user|version|visible|
+---+----+--------------------+-----------+-----------+---+-------+---------+-------------------+--------+------------------+-------+-------+
|  1|node|{communication:mi...| 42.7957187| 13.5690032| []|     []|124176968|2022-07-28 09:47:39|   29598|             owene|     33|   true|
|  2|node|{name -> Центр Кр...| 59.7717926| 30.3261100| []|     []|133444398|2023-03-08 14:57:22|18653914|Maison des Castors|     43|   true|
|  3|node|{amenity -> kinde...|-34.8140751|-58.4899743| []|     []|124176968|2022-07-28 09:47:39|   29598|             owene|     13|   true|
| 10|node|{alt_name -> Mama...| 14.2769353|-11.0519163| []|     []|124176968|2022-07-28 09:47:39|   29598|             owene|     14|   true|
| 54|n

In [12]:
val nodes = (
  planet
    .where(
      $"type" === "node" &&
      $"lon".between(-122.45, -122.25) &&
      $"lat".between(47.5, 47.7)
    )
    .withColumn("geom", st_makePoint($"lon", $"lat"))
    .selectExpr("id as node_id", "geom")
)

nodes: org.apache.spark.sql.DataFrame = [node_id: bigint, geom: point]



In [6]:
nodes.printSchema()

root
 |-- node_id: long (nullable = true)
 |-- geom: point (nullable = true)




In [7]:
nodes.show(truncate=false)

+--------+-------------------------------+
|node_id |geom                           |
+--------+-------------------------------+
|29445653|POINT (-122.3196898 47.64248)  |
|29445655|POINT (-122.3211311 47.6421717)|
|29445656|POINT (-122.3218423 47.6417832)|
|29445657|POINT (-122.3220413 47.6415965)|
|29445659|POINT (-122.3221887 47.6414031)|
|29445660|POINT (-122.322323 47.6411219) |
|29445661|POINT (-122.3223635 47.64096)  |
|29445662|POINT (-122.322506 47.6392676) |
|29445663|POINT (-122.3226326 47.6385371)|
|29447666|POINT (-122.3045716 47.6442165)|
|29447667|POINT (-122.3052346 47.6440985)|
|29447668|POINT (-122.3062284 47.6438864)|
|29447670|POINT (-122.3089121 47.6433095)|
|29449047|POINT (-122.2813041 47.6443495)|
|29449048|POINT (-122.2780591 47.6438462)|
|29449050|POINT (-122.2794678 47.6439584)|
|29449412|POINT (-122.3166305 47.6429371)|
|29449415|POINT (-122.3140945 47.6431403)|
|29449419|POINT (-122.3105004 47.6432074)|
|29449421|POINT (-122.3091586 47.643335) |
+--------+-

In [8]:
val ways = (
  planet
    .where($"type" === "way" && $"tags.highway".isNotNull)
    .selectExpr("id as way_id", "tags as way_tags", "posexplode(nds.ref) as (idx, node_id)")
)

ways: org.apache.spark.sql.DataFrame = [way_id: bigint, way_tags: map<string,string> ... 2 more fields]



In [9]:
ways.printSchema()

root
 |-- way_id: long (nullable = true)
 |-- way_tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- idx: integer (nullable = false)
 |-- node_id: long (nullable = true)




In [10]:
ways.show(truncate=false)

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+----------+
|way_id|way_tags                                                                                                                                                                                                                          |idx|node_id   |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+----------+
|37    |{abutters -> residential, gritting -> priority_3, highway -> residential, is_in -> Sutton Coldfield, maintenance -> gritting, maxspeed -> 30 mph, name -> Maney Hill Road, note -> gritting addition Oct 2010, postal_code -> B72}|0  |200511  

In [11]:
val referencedWays = (
  ways
    .join(nodes, Seq("node_id"))
    .groupBy("way_id")
    .agg(
      first("way_tags").as("way_tags"),
      transform(
        array_sort(collect_list(struct("idx", "geom"))),
        (elem, _) => elem.getField("geom")
      ).as("geoms")
    )
    .where(size($"geoms") >= 2)
    .select($"way_id", $"way_tags", st_makeLine($"geoms").as("geom"))
)

referencedWays: org.apache.spark.sql.DataFrame = [way_id: bigint, way_tags: map<string,string> ... 1 more field]



In [13]:
referencedWays.printSchema()

root
 |-- way_id: long (nullable = true)
 |-- way_tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- geom: linestring (nullable = true)




In [14]:
referencedWays.show()

+-------+--------------------+--------------------+
| way_id|            way_tags|                geom|
+-------+--------------------+--------------------+
|4708373|{bicycle -> dismo...|LINESTRING (-122....|
|4732918|{bicycle -> desig...|LINESTRING (-122....|
|4921074|{highway -> uncla...|LINESTRING (-122....|
|6338129|{highway -> resid...|LINESTRING (-122....|
|6342970|{highway -> resid...|LINESTRING (-122....|
|6343890|{highway -> terti...|LINESTRING (-122....|
|6344404|{highway -> resid...|LINESTRING (-122....|
|6344882|{bicycle -> yes, ...|LINESTRING (-122....|
|6345441|{highway -> resid...|LINESTRING (-122....|
|6350677|{highway -> uncla...|LINESTRING (-122....|
|6356227|{highway -> resid...|LINESTRING (-122....|
|6358495|{highway -> resid...|LINESTRING (-122....|
|6358779|{highway -> resid...|LINESTRING (-122....|
|6376702|{highway -> resid...|LINESTRING (-122....|
|6379160|{highway -> resid...|LINESTRING (-122....|
|6380768|{highway -> resid...|LINESTRING (-122....|
|6383297|{hi

In [35]:
%stop_session

Stopping session: 63f16f54-3209-4ade-8533-22e253102272
Stopped session.
